In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt

# Read the entire file into a list of lines
with open('/mnt/data/data.csv', 'r') as f:
    lines = f.readlines()

# Initialize empty list to hold the data
data_blocks = []

# Initialize empty dictionary to hold the current block of data
current_block = {'comments': [], 'data': []}

# Iterate through the lines
for line in lines:
    if line.startswith('#'):  # this is a comment
        # If we have started collecting data and we encounter a new comment, 
        # this is the start of a new block of data
        if current_block['data']:
            data_blocks.append(current_block)
            current_block = {'comments': [], 'data': []}
        current_block['comments'].append(line.strip())
        # If the comment contains 'nu_z', extract its value
        if 'nu_z' in line:
            match = re.search(r'nu_z=([\d.]+)', line)
            if match:
                current_block['nu_z'] = float(match.group(1))
    else:  # this is data
        current_block['data'].append(line.strip().split())

# Add the last block of data
if current_block['data']:
    data_blocks.append(current_block)

# Correctly define the column names
columns = ["gate potential (muK)", "conductance (1/h)", "conductance_stdDeviation (1/h)"]

# Initialize a figure for the plots
fig = plt.figure(figsize=(20, 5*len(data_blocks)))

for i, block in enumerate(data_blocks):
    # Convert the block of data into a dataframe
    df_block = pd.DataFrame(block['data'], columns=columns)
    df_block = df_block.apply(pd.to_numeric, errors='coerce')

    # Scatter plot of conductance vs gate potential
    ax1 = fig.add_subplot(len(data_blocks), 3, i*3+1)
    ax1.scatter(df_block["gate potential (muK)"], df_block["conductance (1/h)"])
    ax1.set_title(f'Block {i+1} Scatter Plot (nu_z={block["nu_z"]}kHz)')
    ax1.set_xlabel('Gate Potential (muK)')
    ax1.set_ylabel('Conductance (1/h)')

    # Line plot of conductance vs gate potential
    ax2 = fig.add_subplot(len(data_blocks), 3, i*3+2)
    ax2.plot(df_block["gate potential (muK)"], df_block["conductance (1/h)"])
    ax2.set_title(f'Block {i+1} Line Plot (nu_z={block["nu_z"]}kHz)')
    ax2.set_xlabel('Gate Potential (muK)')
    ax2.set_ylabel('Conductance (1/h)')

    # Histogram of conductance
    ax3 = fig.add_subplot(len(data_blocks), 3, i*3+3)
    ax3.hist(df_block["conductance (1/h)"], bins=10)
    ax3.set_title(f'Block {i+1} Histogram (nu_z={block["nu_z"]}kHz)')
    ax3.set_xlabel('Conductance (1/h)')
    ax3.set_ylabel('Frequency')

# Display the plots
plt.tight_layout()
plt.show()